In [ ]:
import typing as t

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## A "network" for multi-class classification

We saw in the previous lab that the two main problems with the perceptron are that it will converge only if the classes are linearly separable and that it can only be used for binary classification.

In this section we are going to create a "network" where the inputs are directly connected to the output layer, in which we have one output unit for each class.

Let's define a *linear layer* as:

$$ \mathbf{z} = W \mathbf{x} + b $$

With $W \in \mathbb{R}^{C \times M} $. $M$ is the dimension of the input and $C$ the number of output units (= number of classes), $\mathbf{x} \in \mathbb{R}^M$ and $b\in \mathbb{R}^C$.

The Softmax function maps a vector $z \in \mathbb{R}^C$ to a vector $q \in \mathbb{R}^C$  such that:

$$ q_i(\mathbf{z}) = \frac{e^{z_i}}{\sum_{j \in {\{1, \ldots, C\}}}{e^{z_j}}} \forall i \in {\{1, \ldots, C\}} = Softmax(\mathbf{z})_i $$

Note that: $ 0 \leq q_i \leq 1 \forall i \in {\{1, \ldots, C\}} $, and: $\sum_{i \in {\{1, \ldots, C\}}}{q_i} = 1 $; therefore we can interpret the Softmax function as a function that can normalize any real vector $\mathbf{z}$  into a probability distribution $\mathbf{q}$ over the $C$ values.

With this setup, a "forward pass" corresponds to calculating $\mathbf{z}$ and applying the Softmax to it.

The implementation of the Softmax and a forward pass is as follows:

In [ ]:
def softmax(z: np.ndarray) -> np.ndarray:
    """
    Takes the output of a layer as input and returns a probability distribution
    input:
        z (np.array)

    returns:
        out (np.array)
    """
    # z_stable = z - np.max(z, axis=0, keepdims=True)  # subtract max per column (use this if you have inf errors)
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)


def forward(W: np.ndarray, b: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Computes the forward pass and returns the softmax
    input:
        W (np.array): (N_CLASSES, INPUT_SHAPE) The weight matrix of the perceptron
        b (np.array): (N_CLASSES, 1) The bias matrix of the perceptron
        x (np.array): (INPUT_SHAPE, 1) The input of the perceptron

    returns:
        (np.array) (C, 1)
    """
    z = W @ x + b

    return softmax(z)

In a supervised classification task, we typically use the cross-entropy function on top of the softmax output as a loss function. We use a 1-hot encoded vector for the true distribution $\mathbf{p}(x)$ , where we have $1$ in the position corresponding to the true label $y$ and $0$ elsewhere:

$$
p_i(x) = \left\{
  \begin{array}{ll}
    1, & \text{if}\ y=i \\
    0, & \text{otherwise}
  \end{array}\right.
$$

and the output of the softmax function as our $\mathbf{q}$.

For a single sample $x$, the cross-entropy loss value for these p(x) and q(x) is then:
$$H(p,q)= − \sum_{i=1}^C{p_i(x)\log q_i(x)}$$

Given that the only non-zero element of the 1-hot vector $p(x)$ is at the $y$ index of the correct class, in practice the $p(x)$ vector is a selector for the $y$ index in the $q(x)$ vector. Therefore, the loss function for a single sample becomes:

$$ Loss = -\log(q_y) = - \log \left( \frac{e^{z_y}}{\sum_{j \in {\{1, \ldots, C\}}}{e^{z_j}}} \right) = - z_y + \log \sum_{j}{e^{z_j}}$$

> **Ques 1**: Derive the gradients of loss with respect to $W$ and $b$.

$$\nabla_W loss = ?$$
$$\nabla_b loss = ?$$



> **Task 1**: Implement the gradients and the loss

In [ ]:
def compute_grads(
    softmaxed: np.ndarray, x: np.ndarray, y: np.ndarray
) -> tuple[np.ndarray, np.ndarray]:
    """
    inputs:
        softmaxed (np.array): (N_CLASSES, batch_size) result of the forward pass
        y (np.array): (N_CLASSES, batch_size) One-hot encoded vector of the label
        x (np.array): (INPUT_SHAPE, batch_size) Input vector

    returns:
        d_W (np.array): (N_CLASSES, INPUT_SHAPE) Gradient of the loss with respect to the weight matrix
        d_b (np.array): (N_CLASSES, batch_size) Gradient of the loss with respect to the bias matrix
    """

    


def compute_loss(softmaxed: np.ndarray, y: np.ndarray) -> float:
    """
    loss for a single datapoint
    inputs:
        softmaxed (np.array): (N_CLASSES, batch_size)
        y (np.array): (N_CLASSES, batch_size)

    returns:
        loss value (float) - averaged over a batch
    """
    
    return loss
    

Now, let's generate some data:

In [ ]:
X, Y = make_blobs(n_samples=500, n_features=2, random_state=42)

plt.figure(figsize=(5, 5))
plt.scatter(X[:, 0], X[:, 1], c=Y)
plt.show()

# encode output as one-hot
Y = OneHotEncoder(categories="auto").fit_transform(Y.reshape(-1, 1)).toarray()

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, random_state=42, test_size=0.4
)

The following function trains our model:

In [ ]:
def train_network(
    X_train, Y_train, X_test, Y_test, lr, n_epochs=10, batch_size=10, random_seed=42
):

    INPUT_SHAPE = X_train.shape[1]
    N_CLASSES = Y_train.shape[1]

    # Initialise metrics lists
    loss_history = []
    acc_history = []
    test_acc_history = []

    # Initialisation of the weights
    np.random.seed(random_seed)
    W = np.random.normal(size=(N_CLASSES, INPUT_SHAPE))
    b = np.random.normal(size=(N_CLASSES, 1))

    n_samples = X_train.shape[0]
    n_batches = int(np.ceil(n_samples / batch_size))

    for epoch in range(n_epochs):
        # Shuffle dataset
        idx = np.random.permutation(n_samples)
        X_train = X_train[idx]
        Y_train = Y_train[idx]

        epoch_loss = []
        epoch_acc = []

        for b in range(n_batches):
            # Mini-batch
            start = b * batch_size
            end = min((b + 1) * batch_size, n_samples)

            X_batch = X_train[start:end].T  # (INPUT_SHAPE, batch_size)
            Y_batch = Y_train[start:end].T  # (N_CLASSES, batch_size)

            # Forward pass
            out = forward(W, b, X_batch)
            softmaxed = softmax(out)

            # Weight update
            d_W, d_b = compute_grads(softmaxed, X_batch, Y_batch)
            W -= lr * d_W
            b -= lr * d_b

            # Batch metrics
            epoch_loss.append(compute_loss(softmaxed, Y_batch))
            epoch_acc.append(
                np.mean(np.argmax(softmaxed, axis=0) == np.argmax(Y_batch, axis=0))
            )

        # --- End of epoch metrics ---
        avg_loss = np.mean(epoch_loss)
        avg_acc = np.mean(epoch_acc)
        loss_history.append(avg_loss)
        acc_history.append(avg_acc)

        # Test accuracy
        out_test = forward(W, b, X_test.T)
        softmaxed_test = softmax(out_test)
        avg_test_acc = np.mean(
            np.argmax(softmaxed_test, axis=0) == np.argmax(Y_test, axis=1)
        )
        test_acc_history.append(avg_test_acc)

        # print(f"Epoch {epoch+1}/{n_epochs} - "
        #      f"Train Loss: {avg_loss:.4f}, "
        #      f"Train Acc: {avg_acc:.4f}, "
        #      f"Test Acc: {avg_test_acc:.4f}")

    return W, b, loss_history, acc_history, test_acc_history

Now, let's train the model on the data and observe the results:

In [ ]:
W, b, loss, acc, test_acc = train_network(
    X_train, Y_train, X_test, Y_test, lr=0.01, n_epochs=10
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.plot(loss)
ax1.set_title("Training Loss")
ax1.set_xlabel("Iterations")
ax2.plot(test_acc, c="r", label="test")
ax2.plot(acc, c="b", label="train")
ax2.set_title("Train and test accuracy")
ax2.set_xlabel("Iterations")
plt.legend()
plt.show()


Now we can visualize the decision boundaries:

In [ ]:
def plot_decision(X: np.ndarray, Y: np.ndarray, forward_fun: t.Callable, ax=None):
    """
    Plot hard decision regions and scatter data points.

    Parameters
    ----------
    X : np.ndarray, shape (N, 2)
        Features
    Y : np.ndarray, shape (N, C)
        One-hot encoded labels
    forward_fun : function
        Callable that accepts x (shape: (2,1)) and outputs probabilities (C,)
    ax : matplotlib Axes, optional
        Axis to plot on (useful for subplots). If None, a new figure is created.
    """

    # --- Meshgrid over feature space ---
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(
        np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200)
    )

    # --- Forward pass for all grid points ---
    grid_points = np.c_[xx1.ravel(), xx2.ravel()]
    probs = np.array([forward_fun(pt.reshape(-1, 1)).ravel() for pt in grid_points])

    # Hard class assignment
    Z = np.argmax(probs, axis=1).reshape(xx1.shape)

    # --- Create axis if none provided ---
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    # Decision surface
    ax.pcolormesh(xx1, xx2, Z, cmap="viridis", shading="auto", alpha=0.6)

    # Data points
    markers = [".", "*", "D", "o", "s", "x"]
    for c in range(Y.shape[1]):
        ax.scatter(
            X[np.argmax(Y, axis=1) == c, 0],
            X[np.argmax(Y, axis=1) == c, 1],
            c="k",
            marker=markers[c % len(markers)],
            label=f"class {c}",
        )

    ax.set_xlabel("$x_1$")
    ax.set_ylabel("$x_2$")
    ax.legend()
    return ax

In [ ]:
plot_decision(X_train, Y_train, lambda x: forward(W, b, x))

The data we used was linearly separable, so let's take a look at how our network would perform on a different dataset: 

In [ ]:
X, Y = make_moons(n_samples=500, noise=0.2)
plt.figure(figsize=(10, 10))
plt.scatter(X[:, 0], X[:, 1], c=Y)
plt.show()
Y = OneHotEncoder(categories="auto").fit_transform(Y.reshape(-1, 1)).toarray()

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, random_state=42, test_size=0.4
)

In [ ]:
W, b, loss, acc, test_acc = train_network(
    X_train, Y_train, X_test, Y_test, 0.01, n_epochs=20
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.plot(loss)
ax1.set_title("Training Loss")
ax1.set_xlabel("Iterations")
ax2.plot(test_acc, c="r", label="test")
ax2.plot(acc, c="b", label="train")
ax2.set_title("Train and test accuracy")
ax2.set_xlabel("Iterations")
plt.legend()
plt.show()
plot_decision(X_test, Y_test, lambda x: forward(W, b, x))

# A "real" neural network

Actually our "network" until now was nothing else than multinomial Logistic Regression. Therefore, to obtain a non-linear decision boundary, we need at least one **hidden layer** with a non-linear activation function; in this case we are going to study a case in which the activation chosen for the hidden layer is the $ReLU$ function:

$$ z = W x + b_1 $$

$$ h = ReLU(W x + b_1) = ReLU(z) $$

We will use the Softmax seen in the previous case as the output layer:

$$ \theta = U h + b_2 $$

$$ \hat{y} = Softmax(\theta) $$

With $U \in \mathbb{R}^{C \times H}$ and $W \in \mathbb{R}^{H \times M}$ and $b_1$, $b_2$ vectors of corresponding dimensions. $H$ is the number of hidden units.

The $ReLU$ function is defined as:

$$ ReLU(x) = 
\begin{cases}
0 \textrm{ if } x < 0\\
x \textrm{ otherwise }
\end{cases}
$$

and it's immediate to observe that the derivative of the ReLU function is: $sgn(ReLU(x)) $.

> **Task 2**: Implement the ReLU activation function, the derivative of the relu, and the forward pass for the neural network.

In [ ]:
def relu(x: np.ndarray) -> np.ndarray:
    """
    input:
        x (np.array)

    returns:
        x (np.array)
    """
    return np.clip(x, 0, np.inf)


def d_relu(x: np.ndarray) -> np.ndarray:
    """Computes the derivative of the relu
    input:
        x (np.array)

    returns:
        x (np.array)
    """
    return (x > 0).astype(int)


def forward_NN(
    U: np.ndarray, b2: np.ndarray, W: np.ndarray, b1: np.ndarray, x: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Forward pass of a two layer perceptron with relu activation
    input:
        W (np.array): (HIDDEN_SHAPE, INPUT_SHAPE) The weight matrix of the hidden layer
        U (np.array): (N_CLASSES, HIDDEN_SHAPE) The weight matrix of the output layer
        b1 (np.array): (HIDDEN_SHAPE, 1) The bias matrix of the hidden layer
        b2 (np.array): (N_CLASSES, 1) The bias matrix of the output layer
        x (np.array): (INPUT_SHAPE, 1) The input of the perceptron

    returns:
        softmaxed (np.array): (N_CLASSES, 1) the output of the network after final activation
        hidden (np.array): (HIDDENT_SHAPE, 1) the output of the hidden layer after activation
        out (np.array): (N_CLASSES, 1) the output of the network before final activation
    """
    hidden = relu(W @ x + b1)
    out = U @ hidden + b2

    return softmax(out), hidden, out

Putting all of the above together, we can see that:


$$ q_i(\mathbf{\theta}) = \frac{e^{\theta_i}}{\sum_{j \in {\{1, \ldots, C\}}}{e^{\theta_j}}} \forall i \in {\{1, \ldots, C\}} = Softmax(\mathbf{\theta})_i = \hat{y}_i$$

and therefore the loss for a single datapoint is:

$$
Loss = -\log(\hat{y}_y) = - \log \left( \frac{e^{\theta_y}}{\sum_{j \in {\{1, \ldots, C\}}}{e^{\theta_j}}} \right) = - \theta_y + \log \sum_{j}{e^{\theta_j}}$$

> **Ques 2** : Derive the gradients of the loss with respect to $W$, $U$, $b_1$ and $b_2$.



> **Task 3**: Based on the previous result implement the computation of the gradients.

In [ ]:
def compute_grads_NN(
    hidden: np.ndarray,
    softmaxed: np.ndarray,
    U: np.ndarray,
    X_batch: np.ndarray,
    Y_batch: np.ndarray,
    W: np.ndarray,
    b1: np.ndarray,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Computes gradients for a two-layer NN with ReLU, batch-aware.

    input:
        hidden (np.array): (HIDDEN_SHAPE, batch_size) hidden layer activations
        softmaxed (np.array): (N_CLASSES, batch_size) output after softmax
        U (np.array): (N_CLASSES, HIDDEN_SHAPE) weights of output layer
        X_batch (np.array): (INPUT_SHAPE, batch_size) batch of inputs
        Y_batch (np.array): (N_CLASSES, batch_size) batch of targets
        W (np.array): (HIDDEN_SHAPE, INPUT_SHAPE) weights of hidden layer
        b1 (np.array): (HIDDEN_SHAPE, 1) biases of hidden layer

    returns:
        d_U (np.array): (N_CLASSES, HIDDEN_SHAPE) gradient wrt U
        d_b2 (np.array): (N_CLASSES, 1) gradient wrt b2
        d_W (np.array): (HIDDEN_SHAPE, INPUT_SHAPE) gradient wrt W
        d_b1 (np.array): (HIDDEN_SHAPE, 1) gradient wrt b1
    """
    

    return d_U, d_b2, d_W, d_b1

The following function trains the network:

In [ ]:
def train_NN(
    X_train,
    Y_train,
    X_test,
    Y_test,
    lr,
    n_hidden,
    forward_NN,
    compute_grads_NN,
    n_epochs=100,
    batch_size=10,
    random_seed=42,
):

    HIDDEN_SHAPE = n_hidden
    INPUT_SHAPE = X_train.shape[1]
    N_CLASSES = Y_train.shape[1]

    # Initialise metrics lists
    loss_history = []
    acc_history = []
    test_acc_history = []

    # Initialisation of the weights
    np.random.seed(random_seed)
    b1 = np.random.normal(size=(HIDDEN_SHAPE, 1))
    W = np.random.normal(size=(HIDDEN_SHAPE, INPUT_SHAPE))
    b2 = np.random.normal(size=(N_CLASSES, 1))
    U = np.random.normal(size=(N_CLASSES, HIDDEN_SHAPE))

    n_samples = X_train.shape[0]
    n_batches = int(np.ceil(n_samples / batch_size))

    for epoch in range(n_epochs):
        # Shuffle dataset
        idx = np.random.permutation(n_samples)
        X_train = X_train[idx]
        Y_train = Y_train[idx]

        epoch_loss = []
        epoch_acc = []

        for b in range(n_batches):
            # Mini-batch
            start = b * batch_size
            end = min((b + 1) * batch_size, n_samples)
            X_batch = X_train[start:end].T  # (INPUT_SHAPE, batch_size)
            Y_batch = Y_train[start:end].T  # (N_CLASSES, batch_size)

            # Forward pass
            softmaxed, hidden, _ = forward_NN(U, b2, W, b1, X_batch)

            # Backprop
            d_U, d_b2, d_W, d_b1 = compute_grads_NN(
                hidden, softmaxed, U, X_batch, Y_batch, W, b1
            )

            # Update
            b1 -= lr * d_b1
            W -= lr * d_W
            b2 -= lr * d_b2
            U -= lr * d_U

            # Batch metrics
            epoch_loss.append(compute_loss(softmaxed, Y_batch))
            epoch_acc.append(
                np.mean(np.argmax(softmaxed, axis=0) == np.argmax(Y_batch, axis=0))
            )

        # --- End of epoch metrics ---
        avg_loss = np.mean(epoch_loss)
        avg_acc = np.mean(epoch_acc)
        loss_history.append(avg_loss)
        acc_history.append(avg_acc)

        # Test accuracy
        softmaxed_test, _, _ = forward_NN(U, b2, W, b1, X_test.T)
        avg_test_acc = np.mean(
            np.argmax(softmaxed_test, axis=0) == np.argmax(Y_test, axis=1)
        )
        test_acc_history.append(avg_test_acc)

        # print(f"Epoch {epoch+1}/{n_epochs} - "
        #      f"Train Loss: {avg_loss:.4f}, "
        #      f"Train Acc: {avg_acc:.4f}, "
        #      f"Test Acc: {avg_test_acc:.4f}")

    return U, b2, W, b1, loss_history, acc_history, test_acc_history

Let's train our network:

In [ ]:
U, b2, W, b1, loss, acc, test_acc = train_NN(
    X_train,
    Y_train,
    X_test,
    Y_test,
    lr=0.1,
    n_hidden=16,
    forward_NN=forward_NN,
    compute_grads_NN=compute_grads_NN,
)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Training loss per epoch
ax1.plot(loss, linestyle="-")
ax1.set_title("Training Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")

# Accuracy per epoch
ax2.plot(acc, c="b", label="train", linestyle="-")
ax2.plot(test_acc, c="r", label="test", linestyle="-")
ax2.set_title("Train and Test Accuracy")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy")
ax2.legend()

plt.show()

In [ ]:
plot_decision(X_test, Y_test, lambda x: forward_NN(U, b2, W, b1, x)[0])

Let's verify how the network performs if we modify the size of the hidden layer (2, 4, 8, 16 hidden units):

In [ ]:
plt.figure(figsize=(20, 15))
for i, n_units in enumerate([2, 4, 8, 16]):
    U, b2, W, b1, loss, acc, test_acc = train_NN(
        X_train,
        Y_train,
        X_test,
        Y_test,
        0.1,
        n_units,
        forward_NN=forward_NN,
        compute_grads_NN=compute_grads_NN,
    )
    plot_decision(
        X_test,
        Y_test,
        lambda x: forward_NN(U, b2, W, b1, x)[0],
        plt.subplot(2, 2, i + 1, title="%i units" % n_units),
    )

We would like now to study the effect of using a different activation function. We will change ReLU to the hyperbolic tangent activation (or *tanh*):

$$
tanh(x) = \frac{e^{2x}-1}{e^{2x}+1}
$$

(Note: you may use the Numpy implementation of tanh)

with derivative (necessary to update the gradient computation): $ tanh'(x) =  1 - tanh(x)^2$ 


> **Task 4**: Update the forward pass and gradient computation of our neural network implementation to use the tanh activation function and vizualize the result.

In [ ]:
def forward_tanh(
    U: np.ndarray, b2: np.ndarray, W: np.ndarray, b1: np.ndarray, X_batch: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Forward pass of a two-layer perceptron with tanh activation
    input:
        W (HIDDEN_SHAPE, INPUT_SHAPE)
        U (N_CLASSES, HIDDEN_SHAPE)
        b1 (HIDDEN_SHAPE, 1)
        b2 (N_CLASSES, 1)
        X_batch (INPUT_SHAPE, batch_size)
    returns:
        softmaxed (N_CLASSES, batch_size)
        hidden (HIDDEN_SHAPE, batch_size)
        out (N_CLASSES, batch_size)
    """
    
    
    return softmax(out), hidden, out


def compute_grads_tanh(
    hidden: np.ndarray,
    softmaxed: np.ndarray,
    U: np.ndarray,
    X_batch: np.ndarray,
    Y_batch: np.ndarray,
    W: np.ndarray,
    b1: np.ndarray,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Batch-aware gradients for tanh activation

    input:
        hidden (np.array): (HIDDEN_SHAPE, batch_size) hidden layer activations
        softmaxed (np.array): (N_CLASSES, batch_size) output after softmax
        U (np.array): (N_CLASSES, HIDDEN_SHAPE) weights of output layer
        X_batch (np.array): (INPUT_SHAPE, batch_size) batch of inputs
        Y_batch (np.array): (N_CLASSES, batch_size) batch of targets
        W (np.array): (HIDDEN_SHAPE, INPUT_SHAPE) weights of hidden layer
        b1 (np.array): (HIDDEN_SHAPE, 1) biases of hidden layer

    returns:
        d_U (np.array): (N_CLASSES, HIDDEN_SHAPE) gradient wrt U
        d_b2 (np.array): (N_CLASSES, 1) gradient wrt b2
        d_W (np.array): (HIDDEN_SHAPE, INPUT_SHAPE) gradient wrt W
        d_b1 (np.array): (HIDDEN_SHAPE, 1) gradient wrt b1
    """
    

    return d_U, d_b2, d_W, d_b1

Visualisation of boundary:

In [ ]:
plt.figure(figsize=(20, 15))
for i, n_units in enumerate([2, 4, 8, 16]):
    U, b2, W, b1, loss, acc, test_acc = train_NN(
        X_train,
        Y_train,
        X_test,
        Y_test,
        0.1,
        n_units,
        forward_NN=forward_tanh,
        compute_grads_NN=compute_grads_tanh,
    )
    plot_decision(
        X_test,
        Y_test,
        lambda x: forward_tanh(U, b2, W, b1, x)[0],
        plt.subplot(2, 2, i + 1, title="%i units" % n_units),
    )

plt.show()